In [6]:
import pandas as pd
import requests
import yfinance as yf
import numpy as np
import time
from tqdm import tqdm
import psycopg2
from datetime import datetime, timedelta
DB_HOST = 'localhost'
DB_NAME = 'tw'
DB_USER = 'mini'
DB_PASS = 'buddyrich134'
def get_db_connection():
    conn = psycopg2.connect(host=DB_HOST, dbname=DB_NAME, user=DB_USER, password=DB_PASS)
    return conn


conn = get_db_connection()
cursor = conn.cursor()
uri_pe = "https://www.twse.com.tw/rwd/zh/afterTrading/BWIBBU_d?date=20240809&selectType=ALL"

res = requests.get(uri_pe)
stock_l = res.json()['data']
stock_list = [i[0] for i in stock_l]

"total len of twse stocks: ", len(stock_list)

('total len of twse stocks: ', 1016)

In [ ]:
stocks = []
undone = []
for code in tqdm([code+".TW" for code in stock_list]):
    stock = yf.download(code, interval='5m', start=datetime.now() - timedelta(60), progress=False, ignore_tz=True, rounding=2, threads=10, group_by='column', timeout=1)
    if len(stock) == 0:
        undone.append(code)
        continue
    stock['Open'] = np.round(stock['Open'], 2)
    stock['High'] = np.round(stock['High'], 2)
    stock['Low'] = np.round(stock['Low'], 2)
    stock['Close'] = np.round(stock['Close'], 2)
    stock['Adj Close'] = np.round(stock['Adj Close'], 2)
    stock.reset_index(inplace=True)
    stock['code'] = code
    stocks.append(stock.values.tolist())
res = sum(stocks, [])
cursor.executemany(f'''
INSERT INTO public.stock_price (da, op, hi, lo, cl, adj vol, code)
VALUES (%s, %s, %s, %s, %s, %s, %s, %s)
''', res)
conn.commit()

In [13]:
len(undone)
stocks = []
undone1 = []
for code in tqdm(undone):
    stock = yf.download(code, interval='1m', start=datetime.now() - timedelta(7), progress=False, ignore_tz=True, rounding=2, threads=10, group_by='column', timeout=1)
    if len(stock) == 0:
        undone1.append(code)
        continue
    stock['Open'] = np.round(stock['Open'], 2)
    stock['High'] = np.round(stock['High'], 2)
    stock['Low'] = np.round(stock['Low'], 2)
    stock['Close'] = np.round(stock['Close'], 2)
    stock['Adj Close'] = np.round(stock['Adj Close'], 2)
    stock.reset_index(inplace=True)
    stock['code'] = code
    stocks.append(stock.values.tolist())
res = sum(stocks, [])
cursor.executemany(f'''
INSERT INTO public.stock_1m_price (da, op, hi, lo, cl, adj, vol, code)
VALUES (%s, %s, %s, %s, %s, %s, %s, %s)
''', res)
conn.commit()
undone1

 67%|██████▋   | 2/3 [00:00<00:00,  8.11it/s]
1 Failed download:
['3481.TW']: YFPricesMissingError('$%ticker%: possibly delisted; No price data found  (1m 2024-08-17 15:31:25.532026 -> 2024-08-24 15:31:25+08:00)')
100%|██████████| 3/3 [00:00<00:00,  9.33it/s]


$3481.TW: possibly delisted; No price data found  (1m 2024-08-17 15:31:25.532026 -> 2024-08-24 15:31:25+08:00)


['3481.TW']

# Material

In [7]:
cursor.execute("select distinct code from public.stock_price;")
conn.commit()
res = cursor.fetchall()
len(res)

1018

In [25]:
len(undone)
stocks = []
undone1 = []
code = '^TWII'
stock = yf.download(code, interval='1m', start=datetime.now() - timedelta(7), progress=False, ignore_tz=True, rounding=2, threads=10, group_by='column', timeout=1)
if len(stock) == 0:
    undone1.append(code)
stock['Open'] = np.round(stock['Open'], 2)
stock['High'] = np.round(stock['High'], 2)
stock['Low'] = np.round(stock['Low'], 2)
stock['Close'] = np.round(stock['Close'], 2)
stock['Adj Close'] = np.round(stock['Adj Close'], 2)
stock.reset_index(inplace=True)
stock['code'] = code
stocks.append(stock.values.tolist())
res = sum(stocks, [])
cursor.executemany(f'''
INSERT INTO public.stock_1m_price (da, op, hi, lo, cl, adj, vol, code)
VALUES (%s, %s, %s, %s, %s, %s, %s, %s)
''', res)
conn.commit()